# Introduction
This notebook is for the data visualization of different kinds of graphs air quality researchers are interested in. It's intended for air quality researchers to track the data collection of sensors and study the relationships between different air quality measurements. The data used is collected by sensors placed on different parts of Georgia Tech's campus and is stored in a database that this notebook queries to retrieve relevant data.

## Using This Notebook
* Copy this file and rename it because you'd want to save your graphs and graph inputs. There can't be two people using the same notebook file because one would have to discard the work done by the other person if they want to use the notebook file.
* To open this notebook and run the cells in it, you have three options:
  * Jupyter
    * You can use it on your browser or locally
    * You can connect it to your local file system
    * For more information, check out the link <a href="https://jupyter.org/try">here</a>
  * Google Colab
    * It's primarily used on the browser
    * You can upload your notebook to Google Drive and open it from there by choosing "Open with" and selecting "Google Colaboratory"
    * For more information, check out the link <a href="https://colab.research.google.com/">here</a>.
  * VS Code:
    * Cannot be used on the browser, only locally
    * Unlike the other two options, you'd have to set up the kernel manually to be able to run the cells (that's why I don't recommend it)
* Currently, this notebook supports 3 graphs: sensor activity, diurnal, timeseries.
* For each graph in this notebook, there's a 3-step process to visualize it:
    * Step 1: Query the Database
    * step 2: Transform the Data (some graphs don't have that)
    * step 3: Create the Graph
* For each graph, there are 4 (or 3 if there's no step 2) cells you need to run. The first 3 (or 2) are function definitions that implement each of the steps mentioned above. The last cell is where you specify the paramters of all the functions and call them.
* Important considerations:
  * The first step (querying the database) takes the longest. It could take up to 10 minutes to run depending on the volume of data you're querying. Therefore, I suggest having the function calls to the other 2 steps in their own cell so that you can modify the parameters and run them again quickly as many times as you want.
  * For each graph, you have to specify the correct table that contains information about the sensor and columns you're interestd in. The implementation of the first step (querying the database) doesn't check if you chose the correct table. It's your responsibility to make sure you're selecting the correct table. For example, if you're interested in the `pm1` and `o3` data of the `SN000-037` sensor, you have to specify the table parameter to be `v100_final`. If you're not sure what the correct table is, check the "Database Overview" section.
* To start using the notebook, first run the cell under "Establish Connection" to connect to the database. Then, run the cell under "Imports". Lastly, run all the cells that have function definitions. Once you've done that, you can start visualizing by playing with the last cell under each graph type and modifying the parameters to your liking.
* If you ever get a "Could not establish connection" error or any kind of error that seems like it's related to the database connection, just rerun the cell under "Establish Connection".

## Database Overview
Here are the tables:
* `location`
* `sensor`
* `mod_pm_raw`
* `v100_raw`
* `v100_final`
* `v200_raw`
* `v200_final`

Here are the sensors and their tables:
* mod_pm: ['MOD-PM-00001', 'MOD-PM-00002', 'MOD-PM-00003', 'MOD-PM-00004', 'MOD-PM-00005', 'MOD-PM-00006', 'MOD-PM-00007', 'MOD-PM-00008', 'MOD-PM-00009', 'MOD-PM-00010', 'MOD-PM-00011', 'MOD-PM-00012', 'MOD-PM-00013', 'MOD-PM-00014', 'MOD-PM-00015', 'MOD-PM-00016', 'MOD-PM-00017', 'MOD-PM-00018', 'MOD-PM-00019', 'MOD-PM-00024', 'MOD-PM-00280', 'MOD-PM-00281', 'MOD-PM-00285', 'MOD-PM-00287', 'MOD-PM-00288', 'MOD-PM-00289', 'MOD-PM-00290', 'MOD-PM-00291', 'MOD-PM-00292', 'MOD-PM-00293', 'MOD-PM-00294', 'MOD-PM-00296', 'MOD-PM-00299']
* v100: ['SN000-036', 'SN000-035', 'SN000-037', 'SN000-032']
* v200: ['SN000-123', 'SN000-124', 'SN000-125', 'SN000-121', 'SN000-119', 'SN000-126', 'SN000-127', 'SN000-110', 'SN000-122']

# Setup

## Establish Connection
<u>Warning:</u> make sure you're connected via VPN before proceeding. <br>
If you don't have the VPN client installed on your machine, visit <a href="https://vpn.gatech.edu">vpn.gatech.edu</a> and log in. Then click on "Download Windows/macOS VPN Client".

In [6]:
# import relevant packages
import sqlalchemy
from sshtunnel import SSHTunnelForwarder

# specify database credentials
username = ""
password = ""
private_server_ip = ""
ip = ""
database = ""

# ssh to database
server = SSHTunnelForwarder(
    ssh_address_or_host=(ip, 22),
    ssh_username=username,
    ssh_password=password,
    remote_bind_address=(private_server_ip, 3306)
)

# start ssh server
server.start()
local_port = str(server.local_bind_port)
engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{host}:{port}/{db}".format(user=username, pw=password, host=private_server_ip, port=local_port, db=database))
connection = engine.connect()

In [20]:
# close the connection to the server/database
server.stop() # only run this when you're done, right before closing the notebook (it's OK if you forgot, but it's good practice to close the connection)

## Imports

In [2]:
from typing import List

import pandas as pd
import plotly.express as px

# Start Visualizing!

## Sensor Activity
<u>Description:</u> visualize the change in datapoints collected per month over all the collected datapoints for the given sensor.<br>
<u>Input:</u> sensor serial number. <br>
<u>Output:</u> a histogram of how many datapoints were collected per month.

In [3]:
def query_database_sensor_activity(sensor: str, table: str, show_outputs: bool = True) -> pd.DataFrame:
    """
    Queries the database for all the timestamps in which a datapoint was collected by the given sensor.
    Parameters:
        sensor (str): the sensor ID to query
        table (str): the table that contains information about the given sensor in the database
        show_outputs (bool): whether to print updates about the function
    Returns:
        pd.DataFrame: the query result
    """
    sql_query = "SELECT timestamp FROM iaq.{} WHERE sn = '{}'".format(table, sensor)
    print(f"Executed SQL query:\n{sql_query}\n") if show_outputs else None
    tsdf = pd.read_sql(sql=sql_query, con=connection).sort_values(by='timestamp')
    print("Shape of query result: {}\n".format(tsdf.shape)) if show_outputs else None
    print(f"Sample of query result:\n{tsdf.sample(5)}\n") if show_outputs else None
    return tsdf

In [4]:
def create_graph_sensor_activity(df: pd.DataFrame, sensor: str) -> None:
    """
    Creates the sensor activity graph using the data from the database query result.
    Parameters:
        df (pd.DataFrame): the dataframe containing the sensor activity data, which we got as the database query result
        sensor (str): the sensor ID queried
    Returns:
        None
    """
    fig = px.histogram(df['timestamp'].dt.date, x='timestamp')

    fig.update_traces(
        xbins=dict(size="M1"))
    
    fig.update_layout(
        title='Datapoint Counts per Month for Sensor {}'.format(sensor),
        xaxis_title='Month',
        yaxis_title='Count',
        xaxis=dict(
            dtick='M1'),
        bargap=0.1)
    
    fig.show()

In [ ]:
# specify the parameters
sensor = "SN000-124"
table = "v200_final"
show_outputs = True

# call the functions
query_result = query_database_sensor_activity(sensor, table, show_outputs)
create_graph_sensor_activity(query_result, sensor)

## Diurnal
<u>Description:</u> visualize the diurnal graph of the chosen columns (e.g. `pm1`, `pm10`, `o3`, etc.). <br>
<u>Input:</u> sensor serial number, columns to visualize, start date and end date of period to visualize. <br>
<u>Output:</u> line graph with each hour of the day (0-23) on the horizontal axis and the average of all values in that hour on the vertical axis.

In [8]:
def query_database_diurnal(sensor: str, table: str, columns: List[str], start_date: str, end_date: str, show_outputs: bool = True) -> pd.DataFrame:
    """
    Queries the database for the given columns and timestamp of each datapoint of the given sensor.
    Parameters:
        sensor (str): the sensor ID to query
        table (str): the table that contains information about the given sensor in the database
        columns (List[str]): the columns to query
        start_date (str): the start date of the datapoints to query
        end_date (str): the end date of the datapoints to query
        show_outputs (bool): whether to print updates about the function
    Returns:
        pd.DataFrame: the query result
    """
    sql_query = "SELECT timestamp, {cols} FROM iaq.{table} WHERE sn = '{sn}' AND '{start}' <= timestamp AND timestamp <= '{end}'".format(cols=', '.join(columns), table=table, sn=sensor, start=start_date, end=end_date)
    print(f"Executed SQL query:\n{sql_query}\n") if show_outputs else None
    didf = pd.read_sql(sql=sql_query, con=connection)
    print(f"Shape of query result: {didf.shape}\n") if show_outputs else None
    print(f"Sample of query result:\n{didf.sample(5)}\n") if show_outputs else None
    return didf

In [10]:
def transform_data_diurnal(df: pd.DataFrame, show_outputs: bool = True) -> pd.DataFrame:
    """
    Transforms the query result into a dataframe that contains the mean of each column for each hour of the day.
    Parameters:
        df (pd.DataFrame): the dataframe containing the query result
        show_outputs (bool): whether to print updates about the function
    Returns:
        pd.DataFrame: the transformed dataframe
    """
    df['hour'] = df['timestamp'].dt.hour
    df = df.groupby(pd.Grouper(key='hour')).mean().reset_index()
    print(f"Sample of dataframe after transformation:\n{df.sample(5)}\n") if show_outputs else None
    return df

In [14]:
def create_graph_diurnal(df: pd.DataFrame, columns: List[str], sensor: str, colors: List[str] = None, dx: int = 5, show_dots: bool = True) -> None:
    """
    Creates the diurnal graph using the trasformed database query result.
    Parameters:
        df (pd.DataFrame): the dataframe containing the diurnal data, which we got by transforming the database query result
        columns (List[str]): the columns to plot
        sensor (str): the sensor ID queried
        colors (List[str]): the colors to use for each column (in the same order as the columns).
            Colors can be specified as names of colors or hex values. If None, the default colors will be used.
        dx (int): the x-axis tick interval
        show_dots (bool): whether to show dots on the graph
    Returns:
        None
    """
    
    fig = px.line(
        df,
        x='hour', 
        y=columns, 
        title=f"Diurnal graph of {', '.join(columns)} for {sensor}",
        color_discrete_sequence=colors if colors else None,
        markers=show_dots)

    fig.update_layout(
        xaxis=dict(dtick=dx))

    fig.show()

In [ ]:
# specify the parameters
sensor = "SN000-037"
table = 'v100_final'
columns = ['pm1', 'pm10', 'pm25', 'o3']
start_date = "2022-08-01"
end_date = "2022-08-03"
show_outputs = True
colors = ['black', 'blue', 'red', 'brown']
dx = 2 
dots_on_datapoints = True

# call the functions
query_result = query_database_diurnal(sensor, table, columns, start_date, end_date, show_outputs)
transformed_result = transform_data_diurnal(query_result, show_outputs)
create_graph_diurnal(transformed_result, columns, sensor, colors, dx, dots_on_datapoints)

## Timeseries
<u>Description:</u> visualize a timeseries graph of the collected datapoints of the chosen columns by the given sensor. <br>
<u>Input:</u> sensor serial number, columns to plot. <br>
<u>Output:</u> a timeseries graph with one plot or more.

In [17]:
def query_database_timeseries(sensor: str, table: str, columns: List[str], start_date: str, end_date: str, show_outputs: bool = True) -> pd.DataFrame:
    """
    Queries the database for the given columns and timestamp of each datapoint of the given sensor.
    Parameters:
        sensor (str): the sensor ID to query
        table (str): the table that contains information about the given sensor in the database
        columns (List[str]): the columns to query
        start_date (str): the start date of the datapoints to query
        end_date (str): the end date of the datapoints to query
        show_outputs (bool): whether to print updates about the function
    Returns:
        pd.DataFrame: the query result
    """
    sql_query = "SELECT timestamp, {cols} FROM iaq.{table} WHERE sn = '{sn}' AND '{start}' <= timestamp AND timestamp <= '{end}'".format(cols=', '.join(columns), table=table, sn=sensor, start=start_date, end=end_date)
    print(f"Executed SQL query:\n{sql_query}\n") if show_outputs else None
    tsdf = pd.read_sql(sql=sql_query, con=connection)
    print(f"Shape of query result: {tsdf.shape}\n") if show_outputs else None
    print(f"Sample of query result:\n{tsdf.sample(5)}\n") if show_outputs else None
    return tsdf

In [16]:
def create_graph_timeseries(df: pd.DataFrame, columns: List[str]):
    """
    Creates the timeseries graph using the data from the database query result.
    Parameters:
        df (pd.DataFrame): the dataframe containing the timeseries data, which we got as the database query result
        columns (List[str]): the columns to plot
    Returns:
        None
    """
    fig = px.line(df, x='timestamp', y=columns)
    fig.show()

In [ ]:
# specify the parameters
sensor = "SN000-036"
table = "v100_final"
columns = ["pm1", "pm10"]
start_date = "2022-08-01"
end_date = "2022-08-03"
show_outputs = True

# call the functions
query_result = query_database_timeseries(sensor, table, columns, start_date, end_date, show_outputs)
create_graph_timeseries(query_result, columns)

# Exporting Graphs

Hover over to the top left side of the plot, you'll see a camera icon that if you hover over will say "Download plot as a png". Click on it.

# Release Notes

This is the first version of this notebook, so there are no release notes. However, future version will have notes that list out all the changes made.